# Build and train a simple model

This is a notebook that builds and trains a simple model as an example of how to use Sparana. It uses ReLu activations with a linear final layer, Xavier initialization and the Adam optimizer. The layers, initialization and optimizer are interchangable with some other types in the library. You will need MNIST pickle files to run this.

In [3]:
# Numpy and cupy are imported, because I use them in all of my experiments, you don't need them to run these cells
import numpy as np
import cupy as cp

# This is needed to load the MNIST files for training and testing
import pickle

# These are the Sparana objects needed to build a model
from sparana.model import model
from sparana.layers import full_relu_layer
from sparana.layers import full_linear_layer
# This is the optimizer used to train the model
from sparana.optimizer import adam_optimizer
# Load the data into this object, which will return optimized minibatches, and track how many minibatches/epochs 
# batches have been loaded
from sparana.data_loader import loader

# This is not needed to train and test a simple model, but I am going to demonstrate it here too
from sparana.saver import model_saver
# Put your own path in here
path = 'path'

# This initializes the model object, the 2 things that are required are the input size, and a list of layer objects.
mymodel = model(input_size = 784, 
                # These are the layers input as a list, the final layer size is the number of classes the model will have.
                layers = [full_relu_layer(size = 1000), 
                          full_relu_layer(size = 800),
                          full_relu_layer(size = 400),
                          full_linear_layer(size = 10)],
                # This is set automatically, but I keep it in here as a demonstration
                comp_type = 'GPU')

# Initialize the weights here, after this randomly generated matrices are now in the GPU memory
mymodel.initialize_weights('Xavier', bias_constant = 0.1)

# Initialize the Adam optimizer, the associated matrices are now in GPU memory
opt = adam_optimizer(mymodel, 0.0001, epsilon = 0.001)

#Initialize the saver
mysaver = model_saver(mymodel)

# Initialize the loader object and load the MNIST dataset from pickle files using pickle
myloader = loader(pickle.load(open('MNIST_train_images.p', 'rb')),
                 pickle.load(open('MNIST_train_labels.p', 'rb')),
                 pickle.load(open('MNIST_test_images.p', 'rb')),
                 pickle.load(open('MNIST_test_labels.p', 'rb')))


Initalizing GPU weights


# Training

This cell just trains, then stores the model on RAM. The saver object can save the model to the hard disk with the line:
```python
mysaver.pickle_model('filename.p')
```
Demonstrated in the Demo-Lobotomizer notebook where I have a practical reason to do that. 

In [4]:

for i in range(20000):
    images, labels = myloader.minibatch(250)
    opt.train_step(images, labels)    

print(mymodel.get_accuracy(myloader.test_data(), myloader.test_labels()))

mysaver.store_model()

0.9848


# Outputs

Just a couple of lines here displaying outputs of the model class. I am looking at 10 datapoints from the training set. The outputs will look the same as test set outputs, and will not clutter up your screen. 

Quick not here, I am using a linear final layer here, so all of the output values should be either very close to 1 or very close to 0, they do not sum to 1 like they would for a softmax layer. 

In [6]:
images, labels = myloader.minibatch(10)

outputs = mymodel.outputs(images)

print('Model outputs')
print(outputs)

# I didn't put this in the library, it's only 1 line, didn't seem necessary

one_hot = np.argmax(outputs, axis = 1)

print('Argmax of model outputs')
print(one_hot)

#We can compare this to the correct classes, the labels

print('Labels')
print(labels)

# I hope these are all correct, won't look good if they are wrong ¯\_(ツ)_/¯

# Or I can just get the accuracy calling get_accuracy from the model object, with the test set

accuracy = mymodel.get_accuracy(myloader.test_data(), myloader.test_labels())

print('Accuracy')
print(accuracy)

Model outputs
[[ 2.4904758e-03 -2.0734221e-04  1.1540107e-02  1.2681112e-02
   7.2287992e-03  9.5505381e-01 -6.8125054e-03 -4.9916729e-03
   4.7410429e-03  9.7680464e-03]
 [ 5.1029474e-03  5.8564320e-03 -1.1851490e-02  1.0037068e+00
  -4.2976886e-03 -1.1061430e-02 -6.3339844e-03  1.5931986e-02
   1.2010679e-02 -1.2753904e-04]
 [-3.2111704e-03 -1.6523011e-02 -5.8771968e-03 -4.6210736e-03
   1.0040379e+00  3.1093508e-04  1.7267548e-02 -1.7646253e-03
   9.5325261e-03  6.2608197e-03]
 [ 1.2597173e-02 -1.2377575e-03  9.4277769e-01  6.8120211e-03
   1.7710321e-02 -8.4303766e-03  1.0956816e-02  1.3016492e-02
   1.1184365e-02 -6.4533129e-03]
 [ 1.0178781e+00 -2.9369667e-03 -5.4296926e-03 -6.4956993e-03
   5.1478297e-04  2.5436655e-03  6.5911636e-03 -9.3612894e-03
  -1.3830140e-03 -2.1958500e-03]
 [ 9.1752037e-03  9.4192761e-01  2.3422047e-02  2.3322269e-02
  -1.9607097e-03 -6.2354058e-03  7.9550743e-03  1.1833869e-02
  -5.9905648e-04 -3.2844469e-03]
 [ 2.9499829e-03  1.0000089e+00 -1.3918504e-